In [1]:
%load_ext autoreload
%autoreload
import os
import sys
# Must set SNORKELDB before importing SnorkelSession
from snorkel import SnorkelSession
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence

session = SnorkelSession()

In [2]:
#Don't run again!!
'''
doc_preprocessor = TextDocPreprocessor('small_data_pp/small_match_test.txt')

corpus_parser = CorpusParser()
corpus_parser.apply(doc_preprocessor)

print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()
'''

Clearing existing...
Running UDF...
Documents: 1
Sentences: 17


In [2]:
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Documents: 400
Sentences: 95656


In [3]:
from snorkel.models import candidate_subclass


GenePhenoPair = candidate_subclass('GenePhenoPair',['gene', 'pheno'])
#GenePhenoPair = candidate_subclass('GenePhenoPair',['gene'])


In [16]:
all_sents = set()
docs = session.query(Document).order_by(Document.name).all()
for i, doc in enumerate(docs):
    if i > 10:break
    for s in doc.sentences:
        all_sents.add(s)

In [17]:
print len(all_sents)

1672


In [18]:
from revised_extraction import GM, PM
from snorkel.candidates import Ngrams, CandidateExtractor

gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=10)
cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    [gene_ngrams, pheno_ngrams], [GM, PM],
                                    symmetric_relations=False)
#cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    #[gene_ngrams], [GM],
                                    #symmetric_relations=False)

In [19]:
print "Extracting Candidates..."

cand_extractor.apply(all_sents, split=0)
all_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==0).all()
print "Number of dev candidates:", len(all_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 421


In [20]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(all_cands, session, annotator_name = 'gold')

<IPython.core.display.Javascript object>

In [21]:
sv

In [3]:
from extraction import GM, PM
from snorkel.candidates import Ngrams, CandidateSpace, CandidateExtractor
from snorkel.models import Document, Sentence, candidate_subclass
from snorkel.viewer import SentenceNgramViewer

session = SnorkelSession()

GenePhenoPair = candidate_subclass('GenePhenoPair',['gene', 'pheno'])
#GenePhenoPair = candidate_subclass('GenePhenoPair',['pheno'])

gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=10)
cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    [gene_ngrams, pheno_ngrams], [GM, PM],
                                    symmetric_relations=True)
#cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    #[pheno_ngrams], [PM],
                                    #symmetric_relations=False)

all_sents = set()
docs = session.query(Document).order_by(Document.name).all()
countdev=0
for i, doc in enumerate(docs):
    for s in doc.sentences:
        all_sents.add(s)

print "Extracting Candidates..."

cand_extractor.apply(all_sents, split=1)
all_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==1).all()
print "Number of dev candidates:", len(all_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[=                                       ] 0%

KeyboardInterrupt: 

In [22]:
sv = SentenceNgramViewer(all_cands, session, annotator_name = 'gold')

<IPython.core.display.Javascript object>

In [23]:
sv